### Adjust Word Group Repeat With Youtube Link

In [19]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [20]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [21]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

# file extention
file_ext = "Twogram"

In [22]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [23]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 2/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [24]:
path_folder_file = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 2/Result/2-Find Word Group Youtube Link/Turkish_Twogram_With_1000_Word_Selected_Count_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [25]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url,count
0,şu anda,3186,3188,hatta zaten şu anda seni kelepçeleyip götürmem...,_S4ZbD_6iCY,https://www.youtube.com/watch?v=_S4ZbD_6iCY&t=...,7
1,şu anda,725,727,bu onun hayatına bereket kattığınızı gösteriyo...,3xCLc2owEtI,https://www.youtube.com/watch?v=3xCLc2owEtI&t=...,7
2,şu anda,89,91,birbirinden güzel üç bölümü şu anda yayınlandı,fyKLb4s3Jdc,https://www.youtube.com/watch?v=fyKLb4s3Jdc&t=89s,7
3,şu anda,71,72,piyasa değerine sahip yani şu anda ol,cFCGIUH3evQ,https://www.youtube.com/watch?v=cFCGIUH3evQ&t=71s,7
4,şu anda,2627,2628,yoksa bu bir takıntı mıydı bağımlılık mıydı ve...,GimA7_uhyWg,https://www.youtube.com/watch?v=GimA7_uhyWg&t=...,7
...,...,...,...,...,...,...,...
221,belki de,4417,4419,belki de çok hoşlanacağı,O3iT4_8jUVw,https://www.youtube.com/watch?v=O3iT4_8jUVw&t=...,4
222,acele et,353,355,hadi acele et kimse gelmeden çekelim,1WaWFqtUsjU,https://www.youtube.com/watch?v=1WaWFqtUsjU&t=...,4
223,acele et,78,80,acele et tamam aşkım,uzYXX4RgcO0,https://www.youtube.com/watch?v=uzYXX4RgcO0&t=78s,4
224,acele et,2036,2038,acele et çavuş bir gören olmasın,YXMgrpbXFTM,https://www.youtube.com/watch?v=YXMgrpbXFTM&t=...,4


In [26]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

41

In [27]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [28]:
## All word group result convert to join result in one row
#df_result_repeat = pd.DataFrame()
#for word_group in search_list:
#    # for repeat 1
#    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
#    # for repeat 2
#    try:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
#    except:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
#    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
#    # repeat block
#    for i in range(len(df_word_group_search_repeat1)):        
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)
#
#    try:
#        df_link_default_var = df_link_default
#        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
#        if word_time_diff_var < 4:
#            word_time_diff_var2 = 3
#        else:
#            word_time_diff_var2 = word_time_diff_var+1.0
#
#        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
#    except:
#        pass
#
#    for j in range(3):
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
#        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
#
#df_result_repeat.reset_index(drop=True, inplace=True)
#df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
#
#df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time2.xlsx", index=False)
#df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join2.xlsx", index=False)

In [29]:
#df_result_repeat

In [30]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [31]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [32]:
df_all_join_result

,video_id,start_time,end_time,search_string,count
0,"_S4ZbD_6iCY,3xCLc2owEtI,fyKLb4s3Jdc,cFCGIUH3ev...","3186,725,89,71,2627,1171,3289,3289,5,3289,5,32...","3188,727,91,72,2628,1173,3291,3291,8,3291,8,32...",şu anda,7.0
1,"M3l_eHps6Jk,4syqfwNfygw,Whh9HzTIOyE,LJlFdEPEib...","258,795,665,1837,257,600,388,257,5,257,5,257,5","260,796,667,1838,258,602,390,258,8,258,8,258,8",ya da,7.0
2,"LY58Fz665UM,wH4L9NiHHIE,0EC2P7aCb6Y,d3LdJzLdpC...","1054,4709,94,888,486,3341,176,94,5,94,5,94,5","1056,4711,96,890,488,3342,178,96,8,96,8,96,8",tabii ki,7.0
3,"p9GnFrcEy0o,IdLaynfyUOQ,Q3Azp5eUgNE,OeHxe75lMN...","1590,249,6005,9826,4350,5026,76,4350,5,4350,5,...","1592,251,6006,9828,4352,5028,78,4352,8,4352,8,...",o zaman,7.0
4,"HBxB7ytsEus,i4cQkYkr8Lo,8bORUKi26Cc,sAr8atA3tu...","615,2250,2837,3011,36,5511,4476,3011,5,3011,5,...","616,2252,2839,3013,38,5513,4478,3013,8,3013,8,...",iyi olur,7.0
5,"O3Xde6lnWqw,peM8h8O4QlY,W5QgaHvSLxc,Lvfe8WqPhs...","73,2219,337,70,636,44,35,337,5,337,5,337,5","75,2221,339,72,638,46,36,339,8,339,8,339,8",hiçbir şey,7.0
6,"OckqrD0x_r8,TLv5OWfkU8U,i5qd1RMfkRE,rAkxQk_Sim...","4217,5216,1730,211,1007,516,5634,5216,5,5216,5...","4219,5218,1732,213,1009,518,5636,5218,8,5218,8...",gördün mü,7.0
7,"kulZ_sWzBR0,NeMTKw_SZRc,cd885E6bPqA,UKVile_Hpw...","11326,3816,424,2004,5634,284,8132,3816,5,3816,...","11328,3818,426,2006,5636,286,8134,3818,8,3818,...",evet efendim,7.0
8,"oZtkK7Nvcw8,RPJg_lUfHvI,B_PW8vRv9Qo,8BXpozdY_A...","632,1003,190,134,4637,2002,208,4637,5,4637,5,4...","634,1005,191,136,4639,2005,210,4639,8,4639,8,4...",daha fazla,7.0
9,"YN6xuSz86PA,gx2mktN5c4A,Uh9XPrYJeKE,pdJwLlL9GV...","7315,376,817,616,79,170,485,79,5,79,5,79,5","7317,379,819,618,81,172,487,81,8,81,8,81,8",bu harika,7.0


#### Copy Move And Delete

In [34]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_1000_Word_Youtube_Talk_Time_Join_Twogram.xlsx']

In [35]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [36]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [7]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
df

,search_string,start_time,end_time,sentence,video_id,video_url
0,acele et,353,355,hadi acele et kimse gelmeden çekelim,1WaWFqtUsjU,https://www.youtube.com/watch?v=1WaWFqtUsjU&t=...
1,acele et,78,80,acele et tamam aşkım,uzYXX4RgcO0,https://www.youtube.com/watch?v=uzYXX4RgcO0&t=78s
2,acele et,2036,2038,acele et çavuş bir gören olmasın,YXMgrpbXFTM,https://www.youtube.com/watch?v=YXMgrpbXFTM&t=...
3,acele et,2814,2816,hadi acele et dur hele dur,MuqJeFdzXgE,https://www.youtube.com/watch?v=MuqJeFdzXgE&t=...
4,anladın mı,569,571,şimdi vefa borcunun ne olduğunu anladın mı maysa,4HonuuM3jN4,https://www.youtube.com/watch?v=4HonuuM3jN4&t=...
...,...,...,...,...,...,...
249,yardım edin,1822,1824,yardım edin,p9GnFrcEy0o,https://www.youtube.com/watch?v=p9GnFrcEy0o&t=...
250,yardım edin,1888,1891,yardım edin i mdaat,8DtAfW7ig1k,https://www.youtube.com/watch?v=8DtAfW7ig1k&t=...
251,yok mu,73,74,barış hiç kız süper kahraman yok mu sufi var,NzwCFn2VQFM,https://www.youtube.com/watch?v=NzwCFn2VQFM&t=73s
252,yok mu,6245,6247,komşum senin haberin yok mu bütün mahalle çalk...,TtI1sXYlIFk,https://www.youtube.com/watch?v=TtI1sXYlIFk&t=...


In [14]:
df_count = df.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,şu anda,7
1,daha fazla,7
2,iyi olur,7
3,tabii ki,7
4,hiçbir şey,7
5,o zaman,7
6,gördün mü,7
7,evet efendim,7
8,ya da,7
9,bu harika,7


In [15]:
df_count_merge = pd.merge(df, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,sentence,video_id,video_url,count
0,şu anda,3186,3188,hatta zaten şu anda seni kelepçeleyip götürmem...,_S4ZbD_6iCY,https://www.youtube.com/watch?v=_S4ZbD_6iCY&t=...,7
1,şu anda,725,727,bu onun hayatına bereket kattığınızı gösteriyo...,3xCLc2owEtI,https://www.youtube.com/watch?v=3xCLc2owEtI&t=...,7
2,şu anda,89,91,birbirinden güzel üç bölümü şu anda yayınlandı,fyKLb4s3Jdc,https://www.youtube.com/watch?v=fyKLb4s3Jdc&t=89s,7
3,şu anda,71,72,piyasa değerine sahip yani şu anda ol,cFCGIUH3evQ,https://www.youtube.com/watch?v=cFCGIUH3evQ&t=71s,7
4,şu anda,2627,2628,yoksa bu bir takıntı mıydı bağımlılık mıydı ve...,GimA7_uhyWg,https://www.youtube.com/watch?v=GimA7_uhyWg&t=...,7
...,...,...,...,...,...,...,...
249,yardım edin,1822,1824,yardım edin,p9GnFrcEy0o,https://www.youtube.com/watch?v=p9GnFrcEy0o&t=...,2
250,yardım edin,1888,1891,yardım edin i mdaat,8DtAfW7ig1k,https://www.youtube.com/watch?v=8DtAfW7ig1k&t=...,2
251,bu kadar,751,753,i nsanlar uzun yaşadığı için yaşlı nüfusu da f...,9fc5eY1gtU0,https://www.youtube.com/watch?v=9fc5eY1gtU0&t=...,2
252,bu kadar,229,231,ekranlar gibi değil ayarlama işlemi olarak gör...,OSJNrfcXksc,https://www.youtube.com/watch?v=OSJNrfcXksc&t=...,2


In [16]:
df_count_merge["search_string"].nunique()

52

In [17]:
df_count_merge[df_count_merge["count"] == 1]

,search_string,start_time,end_time,sentence,video_id,video_url,count
253,ihtiyacım var,71,73,biliyorsunuz ki yaz mevsimindeyiz ve benim rah...,Wf1rn1FL8hY,https://www.youtube.com/watch?v=Wf1rn1FL8hY&t=71s,1


In [18]:
df_count_merge.to_excel("Turkish_Twogram_With_1000_Word_Selected_Count_Manuel.xlsx", index=False)

In [ ]:
df_word_group_time_loc_manuel = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Count_Manuel.xlsx")
df_word_group_time_loc_manuel.sort_values(by=["count","search_string"],inplace=True, ascending=False)
df_word_group_time_loc_manuel.reset_index(drop=True, inplace=True)
df_word_group_time_loc_manuel

In [ ]:
df_word_group_time_loc_manuel.to_excel("Twogram Threegram Youtube Link Selected_Manuel.xlsx", index=False)